In [2]:

import cv2 as cv
import numpy as np
import glob
import os
from Find_card import find_cards 
from Find_contour import find_number_contour
from Find_Color import find_color

# Card size after normlization
CARD_WIDTH = 400
CARD_RATIO = 1.5
CARD_HEIGHT = 600

# Given an image find the contour of the number
# at the center

# Load dataset
images = []
labels = []
# Iterate through image files in dataset
Dataset_Directory = glob.glob("./Main_Dataset/*")
for data in Dataset_Directory:
  # Load image
  img = cv.imread(data)
  images.append(img)

  # Parse image filename for label
  bname = os.path.basename(data) # only get filename ( remove directory path )
  rname, _ = os.path.splitext(bname) # remove extension

  # Example names:
  #  * ZERO_B
  #  * TWO_B_DRAW
  #  * SKIP_Y
  # 
  # The second part is the color (R,G,B,Y)
  # rest is number (or special)
  name_parts = rname.split("_")
  color = name_parts[1]
  label = "_".join([name_parts[0]]+name_parts[2:])

  # Now color is R,G,B,Y
  # label is ZERO, TWO_DRAW, SKIP, ...
  labels.append([color, label])

# Save dimensions of train data, this will
# be useful later.
train_h, train_w = images[0].shape[:2]
# For each train image, find the card inside it
# and only keep the card image
for i in range(len(images)):
  card_imgs, _ = find_cards(images[i])
  if len(card_imgs) != 1:
    # Multiple or zero cards detected in train data image
    print(f"Incorrect number of cards detected in {Dataset_Directory[i]}!")
  else:
    # Found one image, save it
    images[i] = card_imgs[0]

# Normalize each card image
for i in range(len(images)):
  images[i] = cv.resize(images[i], (CARD_WIDTH, CARD_HEIGHT), interpolation=cv.INTER_AREA)

# Find dominant color each card
dominant_colors = []

for i in range(len(images)):
  dominant_color = find_color(images[i])
  dominant_colors.append(dominant_color)

train_contours = []
# Find the contour of the number in the center
# for each card and save it.
for i in range(len(images)):
  contour = find_number_contour(images[i])
  train_contours.append(contour)

# Open camera, pick the default source
cap = cv.VideoCapture(0) # change the index according to the camera source 
#------------------------------------
# Camera process loop
while cap.isOpened():
  ret, frame= cap.read()
  if not ret:
    break
  # Flip frame vertically, webcam image is mirrored
  frame = cv.flip(frame, 1)

  # Find cards in webcam feed
  card_imgs, card_outlines = find_cards(frame)
  if len(card_outlines) > 0:
    cv.drawContours(frame, np.int0(card_outlines), -1, (0, 0, 255), 4)

  if len(card_imgs) > 0:
    # Normalize size for card from webcam feed
    for i in range(len(card_imgs)):
      card_imgs[i] = cv.resize(card_imgs[i], (CARD_WIDTH, CARD_HEIGHT), interpolation=cv.INTER_AREA)

    colors = []
    numbers = []

    # For each card, determine color and number
    for i in range(len(card_imgs)):
      # Determine image color
      dominant_color = find_color(card_imgs[i])

      # Pick closest in train data
      closest_idx = np.argmin([np.linalg.norm(dominant_color - c) for c in dominant_colors])

      # Determine image number contour
      cnt = find_number_contour(card_imgs[i])

      # If no center contour, output unknown
      if cnt is None:
        predict_num = "UNKNOWN"
      else:
        # Pick closest match in contour
        match_idx = np.argmin([cv.matchShapes(cnt,c,1,0.0) for c in train_contours])
        predict_num = labels[match_idx][1]

      colors.append(labels[closest_idx][0])
      numbers.append(predict_num)

    for i in range(len(card_imgs)):
      x, y = np.int0(card_outlines[i][0])
      cv.putText(frame, colors[i] + " " + numbers[i], (x-10, y-10), cv.FONT_HERSHEY_PLAIN, 3.0, (0, 0, 255), thickness = 2, lineType=cv.LINE_AA)


  cv.imshow("UNO Card detection", frame)


  # Quit on escape
  if cv.waitKey(1) == 27:
    break

cap.release()

Incorrect number of cards detected in ./Main_Dataset/SKIP_B.jpg!
Incorrect number of cards detected in ./Main_Dataset/EIGHT_Y.jpg!
Incorrect number of cards detected in ./Main_Dataset/ONE_G.jpg!
Incorrect number of cards detected in ./Main_Dataset/TWO_Y.jpg!
Incorrect number of cards detected in ./Main_Dataset/THREE_G.jpg!
Incorrect number of cards detected in ./Main_Dataset/ONE_B.jpg!
Incorrect number of cards detected in ./Main_Dataset/SKIP_R.jpg!
Incorrect number of cards detected in ./Main_Dataset/SKIP_G.jpg!
Incorrect number of cards detected in ./Main_Dataset/REV_G.jpg!
Incorrect number of cards detected in ./Main_Dataset/ZERO_Y.jpg!
Incorrect number of cards detected in ./Main_Dataset/FOUR_Y.jpg!
Incorrect number of cards detected in ./Main_Dataset/SEVEN_G.jpg!
Incorrect number of cards detected in ./Main_Dataset/ZERO_G.jpg!
Incorrect number of cards detected in ./Main_Dataset/TWO_Y_DRAW.jpg!
Incorrect number of cards detected in ./Main_Dataset/FOUR_G.jpg!
Incorrect number of ca

2023-04-28 18:51:58.470 Python[94834:1445738] Warning: Window move completed without beginning
/var/folders/nd/zthp_2z97hd318dbt8r7xkrr0000gn/T/ipykernel_94834/188001868.py:92: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  cv.drawContours(frame, np.int0(card_outlines), -1, (0, 0, 255), 4)
/var/folders/nd/zthp_2z97hd318dbt8r7xkrr0000gn/T/ipykernel_94834/188001868.py:125: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  x, y = np.int0(card_outlines[i][0])


: 